# Crime classification in San Francisco

In [1]:
# Run these to install package with pip for the right python install

import sys
#!{sys.executable} -m pip install seaborn

In [2]:
import pandas as pd
import seaborn as sns
#%matplotlib nbagg
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme()
plt.figure(figsize=(15,10))

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [3]:
#Read data

df = pd.read_csv('data/train.csv')
print('data loaded')

data loaded


In [4]:
#Get categories (output to find)

cat = df.Category.unique()
print(cat)
df.drop(['Descript','Resolution','Address'],inplace =True, axis=1)  #data useless, unsignificant
df.drop(df[(df.Y>60)|(df.X>-122)].index, inplace=True)  #outliers
df.head()

['WARRANTS' 'OTHER OFFENSES' 'LARCENY/THEFT' 'VEHICLE THEFT' 'VANDALISM'
 'NON-CRIMINAL' 'ROBBERY' 'ASSAULT' 'WEAPON LAWS' 'BURGLARY'
 'SUSPICIOUS OCC' 'DRUNKENNESS' 'FORGERY/COUNTERFEITING' 'DRUG/NARCOTIC'
 'STOLEN PROPERTY' 'SECONDARY CODES' 'TRESPASS' 'MISSING PERSON' 'FRAUD'
 'KIDNAPPING' 'RUNAWAY' 'DRIVING UNDER THE INFLUENCE'
 'SEX OFFENSES FORCIBLE' 'PROSTITUTION' 'DISORDERLY CONDUCT' 'ARSON'
 'FAMILY OFFENSES' 'LIQUOR LAWS' 'BRIBERY' 'EMBEZZLEMENT' 'SUICIDE'
 'LOITERING' 'SEX OFFENSES NON FORCIBLE' 'EXTORTION' 'GAMBLING'
 'BAD CHECKS' 'TREA' 'RECOVERED VEHICLE' 'PORNOGRAPHY/OBSCENE MAT']


In [6]:
train_dummies = pd.get_dummies(df['DayOfWeek'])
df = pd.concat([df,train_dummies], axis=1)
df.drop('DayOfWeek',inplace =True, axis=1)   #replace days by one-hot features
#df['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)

#train_dummies = pd.get_dummies(df['PdDistrict'])
#df = pd.concat([df,train_dummies], axis=1)
df.drop('PdDistrict',inplace =True, axis=1) # estimated  useless

df.head()

,Dates,Category,PdDistrict,X,Y,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,2015-05-13 23:53:00,WARRANTS,NORTHERN,-122.425892,37.774599,0,0,0,0,0,0,1
1,2015-05-13 23:53:00,OTHER OFFENSES,NORTHERN,-122.425892,37.774599,0,0,0,0,0,0,1
2,2015-05-13 23:33:00,OTHER OFFENSES,NORTHERN,-122.424363,37.800414,0,0,0,0,0,0,1
3,2015-05-13 23:30:00,LARCENY/THEFT,NORTHERN,-122.426995,37.800873,0,0,0,0,0,0,1
4,2015-05-13 23:30:00,LARCENY/THEFT,PARK,-122.438738,37.771541,0,0,0,0,0,0,1


In [8]:
#Format time data

df.Dates = pd.to_datetime(df.Dates)
df['Time'] = df.Dates.dt.hour*3600 + df.Dates.dt.minute*60 + df.Dates.dt.second
df['Hour'] = df.Dates.dt.hour
df['Month']=df.Dates.dt.month
df['Year']=df.Dates.dt.year
for key in ['Time','Hour','Month','Year']:
    df[key]=pd.to_numeric(df[key])

#df['Sin_Time'] = np.sin(2 * np.pi * df['Time'] / (24*60*60))  # cycle rpz to explore 
#df['Cos_Time'] = np.cos(2 * np.pi * df['Time'] / (24*60*60))

df['Category'].replace(to_replace=cat,value=[i for i in range(len(cat))],inplace=True)
df.head()

In [10]:
dfsample = df.sample(n=10000)
y=dfsample['Category']
ysample.head()
dfsample.drop('Category',inplace =True, axis=1)
dfsample.drop('Dates',inplace =True, axis=1)
dfsample.head()

5278       6
315627     4
607384     1
710785    13
448284    10
Name: Category, dtype: int64

In [23]:
# Load Library
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier

X=dfsample
y=ysample
# Split the training test set
X_train, X_test, y_train, y_test = train_test_split(dfsample, ysample, test_size=0.2, random_state=42)

#  Fit a Random Forest model
clf = RandomForestClassifier(n_estimators=1000,min_samples_split=50,random_state=42)
clf.fit(X_train, y_train)
p_pred = clf.predict_proba(X_test)
loss=0
count=0
for i in range(len(p_pred)):
    if p_pred[i,np.array(y_test)[i]]<1e-15:
        count+=1
    loss+=np.log(p_pred[i,np.array(y_test)[i]]+1e-15)

print(loss)
print(count)
print(accuracy_score(y_test, y_pred))


-5339.840115681586
6
0.231


In [ ]:
plt.figure(figsize=(15,12))
sns.scatterplot(
    data=dfsample,
    x="X", y="Y", 
    hue="Category",
    legend=False
)

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(
    data=df,
    x="Category",
    stat='density',
    discrete=True,
    hue="Category",
)

In [ ]:
plt.figure(figsize=(15,10))
sns.histplot(
    data = df,
    x = 'Hour',
    discrete=True
)

In [ ]:
sns.displot(
    data = df,
    x = 'Hour',
    hue = 'Category',
    kind='kde'
)

In [ ]:
sns.displot(
    data = dfsample,
    x = 'Sin_Time',
    y = 'Cos_Time'
)

In [ ]:
df.PdDistrict.unique()